In [1]:
# autoimport
%load_ext autoreload
%autoreload 1

In [2]:
import pandas as pd
import joblib
import importlib

from pathlib import Path
from src.gnn_models import *
# import src.gnn_models
#
# importlib.reload(src.gnn_models)
%aimport src.gnn_models

In [4]:
PROCESSED_DATA_DIR = r"E:\gnn_data\processed_step_data_full_node_features"
dataset = FastSTEPDataset(PROCESSED_DATA_DIR, start_index=1)
get_dataset_stats(dataset)

Fast dataset loaded:
- Total samples: 64579
- Processed successfully: 64579
- Failed processing: 77
Label counts in dataset:
Label 1: 40873 instances
Label 0: 23706 instances
Label 1: 63.29% of total instances
Label 0: 36.71% of total instances
Class weights for loss function: [1.5799916815501676, 2.7241626592423858]


In [7]:
dataset[0].x

tensor([[-0.5517, -0.9249, -1.0000,  ..., -0.3974,  0.0397, -0.0896],
        [-0.7946, -0.9850, -0.9998,  ..., -0.2904,  0.2174, -0.0353],
        [-0.8704, -0.9925, -1.0000,  ..., -0.1551,  0.1201, -0.3482],
        ...,
        [-0.8704, -0.9925, -1.0000,  ...,  0.3507, -0.0930, -0.2071],
        [-0.8704, -0.9925, -1.0000,  ...,  0.1529, -0.1725,  0.4640],
        [-0.8704, -0.9925, -1.0000,  ...,  0.1529, -0.1725,  0.4640]])

In [5]:
model = DynamicGNN(
    input_features=dataset[0].x.shape[1],
    hidden_sizes=[512, 512],
    conv_dropout_rate=0.1,
    classifier_dropout_rate=0.0,
    use_layer_norm=True,
    pool_hidden_size=128
)

Created Dynamic GNN model:
- Input features: 22
- Number of hidden layers: 2
- Hidden layer sizes: [512, 512]
- Output classes: 2
- Convolution dropout rate: 0.1
- Classifier dropout rate: 0.0
- Layer normalization: True


C:\Users\phanm\miniconda3\envs\machine-learning\Lib\site-packages\torch_geometric\deprecation.py:26: UserWarning: 'nn.glob.GlobalAttention' is deprecated, use 'nn.aggr.AttentionalAggregation' instead
  warnings.warn(out)


In [12]:
# load model from disk
model_save_path = "dynamic_gnn_model.pth"
if Path(model_save_path).exists():
    print(f"Loading model from {model_save_path}")
    model.load_state_dict(torch.load(model_save_path))
else:
    print(f"Model file {model_save_path} does not exist. Initializing a new model.")

Loading model from dynamic_gnn_model.pth


In [ ]:
torch.cuda.empty_cache()

trained_model, history = simple_train_model_v2(
    dataset,
    gnn_model=model,
    num_epochs=5,
    batch_size=8,
    learning_rate=0.001,
    start_index=0,
    num_graphs_to_use=500,
)

In [18]:
# save model to disk
model_save_path = "dynamic_gnn_model.pth"
torch.save(trained_model.state_dict(), model_save_path)
# save training history
history_save_path = "training_history.pkl"
with open(history_save_path, "wb") as f:
    joblib.dump(history, f)

In [9]:
dataset_trim = FastSTEPDataset(PROCESSED_DATA_DIR, start_index=0, end_index=7)
get_dataset_stats(dataset_trim)

Fast dataset loaded:
- Total samples: 64579
- Processed successfully: 64579
- Failed processing: 77
Label counts in dataset:
Label 1: 40873 instances
Label 0: 23706 instances
Label 1: 63.29% of total instances
Label 0: 36.71% of total instances
Class weights for loss function: [1.5799916815501676, 2.7241626592423858]


In [10]:
dataset_trim[0].x

tensor([[228.0000,  -0.5517,  -0.9249,  ...,  -1.0000,  -0.9971,  -0.7675],
        [287.0000,  -0.7946,  -0.9850,  ...,  -0.9887,  -0.9943,  -1.0000],
        [ 17.0000,  -0.8704,  -0.9925,  ...,  -1.0000,  -0.9971,  -1.0000],
        ...,
        [ 59.0000,  -0.8704,  -0.9925,  ...,  -0.9785,  -0.9938,  -1.0000],
        [146.0000,  -0.8704,  -0.9925,  ...,  -0.9785,  -0.9938,  -1.0000],
        [146.0000,  -0.8704,  -0.9925,  ...,  -0.9785,  -0.9938,  -1.0000]])

In [13]:
model_with_embed = DynamicGNN_Embedding(
    input_features=dataset[0].x.shape[1],
    embedding_dim=16,
    hidden_sizes=[512, 512],
    conv_dropout_rate=0.2,
    classifier_dropout_rate=0.2,
    use_layer_norm=True,
    pool_hidden_size=128
)

Created Dynamic GNN model:
- Input features: 22
- Embedding dimension: 16
- Number of hidden layers: 2
- Hidden layer sizes: [512, 512]
- Output classes: 2
- Convolution dropout rate: 0.2
- Classifier dropout rate: 0.2
- Layer normalization: True


In [15]:
trained_model, history = simple_train_model_v2(
    dataset_trim,
    gnn_model=model_with_embed,
    num_epochs=300,
    batch_size=32,
    learning_rate=0.001,
    start_index=0,
    num_graphs_to_use=50000,
)

Label 1: 346 instances
Label 0: 154 instances
Label 1: 69.20% of total instances
Label 0: 30.80% of total instances
Class weights: tensor([3.2468, 1.4451], device='cuda:0')
Splitting dataset into train and validation sets
Train samples: 400
Validation samples: 100

Starting training for 100 epochs...


Epoch 1/100 [Val]: 100%|██████████| 4/4 [00:00<00:00, 19.11it/s, loss=0.9531, acc=74.00%]


Epoch 1/100 - Train Loss: 0.5530, Acc: 77.00% | Val Loss: 0.6855, Acc: 74.00% (Best Val: 74.00%)


Epoch 2/100 [Val]: 100%|██████████| 4/4 [00:00<00:00, 13.93it/s, loss=0.9515, acc=75.00%]


Epoch 2/100 - Train Loss: 0.5348, Acc: 78.00% | Val Loss: 0.6761, Acc: 75.00% (Best Val: 75.00%)


...

Epoch 98/100 - Train Loss: 0.3052, Acc: 85.75% | Val Loss: 0.3501, Acc: 84.00% (Best Val: 87.00%)


Epoch 99/100 [Val]: 100%|██████████| 4/4 [00:00<00:00,  4.99it/s, loss=0.3400, acc=78.00%]


Epoch 99/100 - Train Loss: 0.3034, Acc: 83.50% | Val Loss: 0.3731, Acc: 78.00% (Best Val: 87.00%)


Epoch 100/100 [Val]: 100%|██████████| 4/4 [00:00<00:00, 18.82it/s, loss=0.3051, acc=82.00%]

Epoch 100/100 - Train Loss: 0.3170, Acc: 85.50% | Val Loss: 0.3469, Acc: 82.00% (Best Val: 87.00%)
Training completed!
Training time: 214.380928999999
Best validation accuracy: 87.00%


In [ ]:
trained_model.embedding